In [1]:
# 1. 필수 라이브러리 임포트
import pandas as pd # 데이터 핸들링용
import numpy as np # 수치 연산용
from sklearn.model_selection import train_test_split # 데이터셋 분할용
from sklearn.preprocessing import StandardScaler # 수치형 데이터 스케일링용

# 2. 신용카드 거래 데이터 로드 (파일 경로 확인 필요)
df = pd.read_csv('creditcard.csv')

# 3. 시간(Time) 데이터를 24시간 주기의 시간대(Hour) 정보로 변환 (패턴 추출)
df['Hour'] = (df['Time'] // 3600) % 24

# 4. 결제 금액(Amount)의 큰 수치 편차를 해결하기 위해 로그 변환(Log Transformation) 적용
df['Log_Amount'] = np.log1p(df['Amount'])

# 5. 모델 학습에 불필요한 원본 컬럼(Time, Amount) 및 정답(Class) 제외하여 피처 세트 구축
X = df.drop(['Class', 'Time', 'Amount'], axis=1)

# 6. 정답 레이블(0: 정상, 1: 사기)만 별도로 분리
y = df['Class']

# 7. 현실적인 성능 평가를 위해 테스트 데이터(20%)를 먼저 분리 (사기 비율 유지)
# 이후 증강 작업은 오직 학습 데이터(X_train, y_train)에만 수행함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 8. DL 모델과 트리 모델 모두에 적합하도록 데이터 표준화(Scaling) 수행
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # 학습 데이터 기준 학습 및 변환
X_test_scaled = scaler.transform(X_test) # 테스트 데이터는 학습 기준에 맞춰 변환

print("✅ 셀 1: 공통 전처리 완료 (X_train_scaled, X_test_scaled 생성됨)")

✅ 셀 1: 공통 전처리 완료 (X_train_scaled, X_test_scaled 생성됨)


In [2]:
# 1. 추가 라이브러리 임포트 (증강 도구)
from imblearn.over_sampling import SMOTE # SMOTE 증강용
from sklearn.cluster import KMeans # K-cGAN 군집화용

# 2. 증강 목표 비율 설정 (정상 데이터의 20% 수준으로 사기 데이터 확보)
target_ratio = 0.2

# --- [방법 A] Original: 증강하지 않은 불균형 원본 학습 데이터 ---
# 변수명 통일을 위해 원본 그대로 복사
X_train_org, y_train_org = X_train_scaled, y_train.values

# --- [방법 B] SMOTE: 선형 보간을 이용한 정석적인 증강 ---
smote = SMOTE(sampling_strategy=target_ratio, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# --- [방법 C] cGAN: 전체 사기 데이터의 통계값(평균/편차)을 이용한 단순 생성 ---
# 실제 사기 데이터의 평균과 표준편차 계산
fraud_indices = np.where(y_train == 1)[0]
fraud_mean = X_train_scaled[fraud_indices].mean(axis=0)
fraud_std = X_train_scaled[fraud_indices].std(axis=0)
# 생성해야 할 개수 산출 (정상의 20% - 현재 사기 수)
needed_cgan = int(len(X_train_scaled[y_train == 0]) * target_ratio) - len(fraud_indices)
# 정규분포를 기반으로 가짜 사기 데이터 생성
fake_cgan = np.random.normal(fraud_mean, fraud_std * 0.25, size=(needed_cgan, X_train_scaled.shape[1]))
# 원본과 결합
X_train_cgan = np.vstack([X_train_scaled, fake_cgan])
y_train_cgan = np.append(y_train.values, np.ones(needed_cgan))

# --- [방법 D] K-cGAN: 군집화(K-Means) 후 각 군집별 개별 생성 (가장 정교함) ---
X_fraud_raw = X_train_scaled[fraud_indices]
# 사기 패턴을 10개로 쪼개어 학습 (세부 특징 보존)
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_fraud_raw)
needed_kcgan = int(len(X_train_scaled[y_train == 0]) * target_ratio) - len(fraud_indices)
gen_per_cluster = needed_kcgan // 10
gen_samples_kcgan = []
for i in range(10): # 각 사기 그룹별로 반복 생성
    cluster_subset = X_fraud_raw[clusters == i]
    fake_subset = np.random.normal(cluster_subset.mean(axis=0), cluster_subset.std(axis=0) * 0.25, size=(gen_per_cluster, X_train_scaled.shape[1]))
    gen_samples_kcgan.append(fake_subset)
# 최종 K-cGAN 데이터셋 결합
X_train_kcgan = np.vstack([X_train_scaled, np.vstack(gen_samples_kcgan)])
y_train_kcgan = np.append(y_train.values, np.ones(len(np.vstack(gen_samples_kcgan))))

# 3. 모든 데이터셋을 딕셔너리에 저장하여 비교 실험 준비 완료
experimental_sets = {
    "Original": (X_train_org, y_train_org),
    "SMOTE": (X_train_smote, y_train_smote),
    "cGAN": (X_train_cgan, y_train_cgan),
    "K-cGAN": (X_train_kcgan, y_train_kcgan)
}

print(f"✅ 셀 2: 4종 데이터 증강 완료!")
for name, (X_tr, y_tr) in experimental_sets.items():
    print(f"- {name}: 데이터 {len(X_tr)}건 (사기 비중: {np.mean(y_tr)*100:.1f}%)")

✅ 셀 2: 4종 데이터 증강 완료!
- Original: 데이터 227845건 (사기 비중: 0.2%)
- SMOTE: 데이터 272941건 (사기 비중: 16.7%)
- cGAN: 데이터 272941건 (사기 비중: 16.7%)
- K-cGAN: 데이터 272935건 (사기 비중: 16.7%)


C:\Users\sunhe\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [3]:
# 1. 필수 라이브러리 임포트 (모델링 및 평가용)
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트 분류기
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score, precision_recall_curve # 평가지표들
import pandas as pd # 결과 정리용

# 2. 각 증강 기법별 결과를 저장할 리스트 초기화
rf_comparison_results = []

# 3. 4종 데이터셋(Original, SMOTE, cGAN, K-cGAN)을 하나씩 꺼내어 실험 반복
for name, (X_tr, y_tr) in experimental_sets.items():
    print(f"🚀 {name} 데이터셋으로 랜덤포레스트 학습 중...")
    
    # 4. 랜덤포레스트 모델 설정 (성능 최적화를 위해 나무 500개 사용)
    # n_jobs=-1은 모든 CPU 코어를 사용하여 연산 속도를 높임
    rf_model = RandomForestClassifier(n_estimators=500, max_features='sqrt', n_jobs=-1, random_state=42)
    
    # 5. 해당 데이터셋으로 모델 학습 수행
    rf_model.fit(X_tr, y_tr)
    
    # 6. 테스트 데이터에 대해 사기(Class 1)일 확률 예측
    y_probs = rf_model.predict_proba(X_test_scaled)[:, 1]
    
    # 7. 정밀도-재현율 곡선(PR Curve)을 통해 모든 임계값에서의 지표 계산
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_probs)
    
    # 8. F1-Score를 최대로 만드는 최적의 임계값(Threshold) 찾기
    # f1 = 2 * (precision * recall) / (precision + recall)
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10) # 0 나누기 방지
    best_idx = np.argmax(f1_scores) # F1이 가장 높은 인덱스 추출
    best_th = thresholds[best_idx] # 해당 지점의 임계값 저장
    
    # 9. 최적 임계값을 적용한 최종 지표 산출 및 결과 저장
    rf_comparison_results.append({
        "Dataset": name,
        "F1-Score": f1_scores[best_idx],
        "Recall": recalls[best_idx],
        "Precision": precisions[best_idx],
        "ROC-AUC": roc_auc_score(y_test, y_probs),
        "Best_Threshold": best_th
    })

# 10. 전체 실험 결과를 데이터프레임으로 변환하여 한눈에 비교
rf_results_df = pd.DataFrame(rf_comparison_results)
print("\n[🏆 랜덤포레스트 데이터셋별 최종 성능 비교]")
print(rf_results_df.sort_values(by="F1-Score", ascending=False).to_string(index=False))

🚀 Original 데이터셋으로 랜덤포레스트 학습 중...
🚀 SMOTE 데이터셋으로 랜덤포레스트 학습 중...
🚀 cGAN 데이터셋으로 랜덤포레스트 학습 중...
🚀 K-cGAN 데이터셋으로 랜덤포레스트 학습 중...

[🏆 랜덤포레스트 데이터셋별 최종 성능 비교]
 Dataset  F1-Score   Recall  Precision  ROC-AUC  Best_Threshold
    cGAN  0.894737 0.867347   0.923913 0.960259           0.404
  K-cGAN  0.892473 0.846939   0.943182 0.967824           0.440
Original  0.888889 0.857143   0.923077 0.961150           0.376
   SMOTE  0.877005 0.836735   0.921348 0.978849           0.616
